<a href="https://www.kaggle.com/code/oscaryezfeijo/spam-ham-prediction?scriptVersionId=178396909" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [2]:
# Importing the dataset
data =pd.read_csv('../input/spam-mails-dataset/spam_ham_dataset.csv')

In [3]:
data.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [4]:
data.tail()

,Unnamed: 0,label,text,label_num
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0
5170,4807,spam,Subject: important online banking alert\r\ndea...,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [6]:
# Extract texts and labels
texts = data['text']
labels = data['label_num']

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer and fit it on the texts
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)  # Assume 'texts' is your list of text data

# Convert texts to sequences and pad them to ensure uniform length
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=200)


In [8]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          640000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 771,713
Trainable params: 771,713
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(X_train, y_train, batch_size=64, epochs=5, validation_data=(X_test, y_test))


Epoch 1/5
65/65 [==============================] - 36s 551ms/step - loss: 0.3338 - accuracy: 0.8392 - val_loss: 0.1893 - val_accuracy: 0.9401
Epoch 2/5
65/65 [==============================] - 36s 560ms/step - loss: 0.0721 - accuracy: 0.9749 - val_loss: 0.0772 - val_accuracy: 0.9681
Epoch 3/5
65/65 [==============================] - 36s 551ms/step - loss: 0.0265 - accuracy: 0.9913 - val_loss: 0.0604 - val_accuracy: 0.9768
Epoch 4/5
65/65 [==============================] - 36s 549ms/step - loss: 0.0147 - accuracy: 0.9964 - val_loss: 0.0547 - val_accuracy: 0.9836
Epoch 5/5
65/65 [==============================] - 36s 553ms/step - loss: 0.0078 - accuracy: 0.9978 - val_loss: 0.0579 - val_accuracy: 0.9836


In [11]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

33/33 [==============================] - 1s 35ms/step - loss: 0.0579 - accuracy: 0.9836
Loss: 0.05793321505188942
Accuracy: 0.9835748672485352


In [12]:
# Assuming the previous steps and the LSTM model are already defined and trained

# Example new email text
new_email = ["Congratulations! You've won a $1000 gift card. Click here to claim now."]

# Preprocess the new email text
new_email_seq = tokenizer.texts_to_sequences(new_email)
new_email_padded = pad_sequences(new_email_seq, maxlen=200)

# Predict using the model
prediction = model.predict(new_email_padded)
print("Spam Probability:", prediction[0][0])

# Interpret the prediction
if prediction[0][0] >= 0.5:
    print("The email is classified as Spam")
else:
    print("The email is classified as Ham")

Spam Probability: 0.99896014
The email is classified as Spam


In [13]:
# Assuming the previous steps and the LSTM model are already defined and trained

# Example new email text
new_email = [" Hi John, just a reminder about our meeting that is scheduled for 3 PM. Please let us know if you need to reschedule, regards Jane."]

# Preprocess the new email text
new_email_seq = tokenizer.texts_to_sequences(new_email)
new_email_padded = pad_sequences(new_email_seq, maxlen=200)

# Predict using the model
prediction = model.predict(new_email_padded)
print("Spam Probability:", prediction[0][0])

# Interpret the prediction
if prediction[0][0] >= 0.5:
    print("The email is classified as Spam")
else:
    print("The email is classified as Ham")

Spam Probability: 0.08969656
The email is classified as Ham


In [14]:
import numpy as np
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the discriminator
def create_discriminator(max_sequence_length, vocab_size):
    discriminator_input = Input(shape=(max_sequence_length,))
    
    # Define the discriminator architecture (e.g., LSTM-based)
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(discriminator_input)
    x = LSTM(units=64)(x)  # LSTM layer with 64 units (adjust as needed)
    
    # Output layer for classifying as spam (1) or ham (0)
    spam_classifier_output = Dense(1, activation='sigmoid')(x)
    
    discriminator_model = Model(discriminator_input, spam_classifier_output)
    return discriminator_model


In [15]:
# Define the discriminator components
max_sequence_length = 100  # Adjust as needed
vocab_size = 10000  # Adjust as needed
embedding_dim = 100  # Adjust as needed

# Create the discriminator
discriminator = create_discriminator(max_sequence_length, vocab_size)

In [16]:
# Compile the discriminator for binary classification
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

In [17]:
# Load labeled dataset
import pandas as pd
your_dataset = pd.read_csv('../input/spam-mails-dataset/spam_ham_dataset.csv')

# Assuming your dataset has a 'text' column for email text and a 'label' column for spam/ham labels
email_text = your_dataset['text'].tolist()
labels = your_dataset['label'].tolist()

In [18]:
# Create a tokenizer and fit it on the email text
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(email_text)

# Convert text to numerical sequences
sequences = tokenizer.texts_to_sequences(email_text)

# Pad sequences to ensure the same length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert labels to NumPy array
labels = np.array(labels)

# Shuffle the data and labels
shuffle_indices = np.random.permutation(len(padded_sequences))
padded_sequences = padded_sequences[shuffle_indices]
labels = labels[shuffle_indices]

In [19]:
# Convert labels to numerical values (0 for ham, 1 for spam)
labels = [1 if label == 'spam' else 0 for label in labels]

# Create a tokenizer and fit it on the email text
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(email_text)

# Convert text to numerical sequences
sequences = tokenizer.texts_to_sequences(email_text)

# Pad sequences to ensure the same length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert labels to NumPy array
labels = np.array(labels)

# Shuffle the data and labels
shuffle_indices = np.random.permutation(len(padded_sequences))
padded_sequences = padded_sequences[shuffle_indices]
labels = labels[shuffle_indices]

In [20]:
# Train the discriminator
batch_size = 32  # Adjust as needed
num_epochs = 10  # Adjust as needed
num_batches = len(padded_sequences) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        # Prepare a batch of real data and labels
        batch_data = padded_sequences[batch * batch_size : (batch + 1) * batch_size]
        batch_labels = labels[batch * batch_size : (batch + 1) * batch_size]
        
        # Train the discriminator on the batch of real data
        discriminator_loss = discriminator.train_on_batch(batch_data, batch_labels)
        
        # Monitor and log the losses and progress as needed
        print(f"Epoch {epoch + 1}, Batch {batch + 1}/{num_batches}, Loss: {discriminator_loss[0]}, Accuracy: {discriminator_loss[1]}")


Epoch 1, Batch 1/161, Loss: 0.6937505006790161, Accuracy: 0.4375
Epoch 1, Batch 2/161, Loss: 0.6922463774681091, Accuracy: 0.59375
Epoch 1, Batch 3/161, Loss: 0.6936545968055725, Accuracy: 0.5
Epoch 1, Batch 4/161, Loss: 0.693144679069519, Accuracy: 0.5625
Epoch 1, Batch 5/161, Loss: 0.6915252804756165, Accuracy: 0.65625
Epoch 1, Batch 6/161, Loss: 0.6932979822158813, Accuracy: 0.53125
Epoch 1, Batch 7/161, Loss: 0.6906138062477112, Accuracy: 0.71875
Epoch 1, Batch 8/161, Loss: 0.6920738220214844, Accuracy: 0.625
Epoch 1, Batch 9/161, Loss: 0.6902031302452087, Accuracy: 0.65625
Epoch 1, Batch 10/161, Loss: 0.6911007165908813, Accuracy: 0.59375
Epoch 1, Batch 11/161, Loss: 0.6853820085525513, Accuracy: 0.78125
Epoch 1, Batch 12/161, Loss: 0.6911554336547852, Accuracy: 0.625
Epoch 1, Batch 13/161, Loss: 0.687399685382843, Accuracy: 0.6875
Epoch 1, Batch 14/161, Loss: 0.6902885437011719, Accuracy: 0.625
Epoch 1, Batch 15/161, Loss: 0.6858815550804138, Accuracy: 0.65625
Epoch 1, Batch 16/1

In [21]:
# Save the trained discriminator weights
discriminator.save_weights('discriminator_weights.h5')

In [22]:
# Sample email texts
sample_email_texts = [
    "Congratulations! You've won a million dollars!",
    "Hi there, I hope you're doing well.",
    "Get a discount on our new products today!",
    "Important: Your account security information",
    "Free trial offer for our premium service"
]

# Tokenize and pad the sample email texts
max_sequence_length = 100  # Assuming the same max_sequence_length used during training
sequences = tokenizer.texts_to_sequences(sample_email_texts)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Use the trained discriminator to classify the emails
predicted_labels = discriminator.predict(padded_sequences)

# Convert predicted labels to 'Spam' or 'Ham'
predicted_classes = ["Spam" if label >= 0.5 else "Ham" for label in predicted_labels]

# Print the results
for i, email in enumerate(sample_email_texts):
    print(f"Email {i + 1}: {email}")
    print(f"Predicted Class: {predicted_classes[i]}")
    print(f"Predicted Score: {predicted_labels[i][0]}")
    print()


Email 1: Congratulations! You've won a million dollars!
Predicted Class: Ham
Predicted Score: 0.18171045184135437

Email 2: Hi there, I hope you're doing well.
Predicted Class: Spam
Predicted Score: 0.5032325983047485

Email 3: Get a discount on our new products today!
Predicted Class: Ham
Predicted Score: 0.4429335296154022

Email 4: Important: Your account security information
Predicted Class: Spam
Predicted Score: 0.5109655261039734

Email 5: Free trial offer for our premium service
Predicted Class: Ham
Predicted Score: 0.299944132566452



In [23]:
# Sample email texts
sample_email_texts = [
    "Congratulations! You've won a million dollars!",
    "Hi there, I hope you're doing well.",
    "Get a discount on our new products today!",
    "Important: Your account security information",
    "Free trial offer for our premium service"
]

# Tokenize and pad the sample email texts
max_sequence_length = 100  # Assuming the same max_sequence_length used during training
sequences = tokenizer.texts_to_sequences(sample_email_texts)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Use the trained discriminator to classify the emails
predicted_labels = discriminator.predict(padded_sequences)

# Convert predicted labels to 'Spam' or 'Ham'
predicted_classes = ["Spam" if label >= 0.5 else "Ham" for label in predicted_labels]

# Print the results
for i, email in enumerate(sample_email_texts):
    print(f"Email {i + 1}: {email}")
    print(f"Predicted Class: {predicted_classes[i]}")
    print(f"Predicted Score: {predicted_labels[i][0]}")
    print()


Email 1: Congratulations! You've won a million dollars!
Predicted Class: Ham
Predicted Score: 0.18171045184135437

Email 2: Hi there, I hope you're doing well.
Predicted Class: Spam
Predicted Score: 0.5032325983047485

Email 3: Get a discount on our new products today!
Predicted Class: Ham
Predicted Score: 0.4429335296154022

Email 4: Important: Your account security information
Predicted Class: Spam
Predicted Score: 0.5109655261039734

Email 5: Free trial offer for our premium service
Predicted Class: Ham
Predicted Score: 0.299944132566452

